In [1]:
import subprocess
import sys
from pathlib import Path
import gdown

WSI_DOWNLOAD_LINK = "https://drive.google.com/uc?id=1noRtbC5fxBlnO7YnvktjIDhFI61PdOSB"
WSI_NAME = Path("TCGA-21-5784-01Z-00-DX1.tif")
OUTPUT_FOLDER = Path('/tmp/')

def download(output_folder, download_link, name):
    output_path = Path(output_folder) / name
    if not output_path.exists():
        gdown.download(download_link, str(output_path))

download(OUTPUT_FOLDER, WSI_DOWNLOAD_LINK, WSI_NAME)

In [2]:
import multiresolutionimageinterface as mir
from wholeslidedata.iterators import create_patch_iterator
import time
TILE_SIZE = 512
LEVEL = 0
SPACING = 0.5
IMAGE_PATH = "/tmp/TCGA-21-5784-01Z-00-DX1.tif"

## MultiResolutionImageInterface

In [3]:
t1_init = time.time()
reader = mir.MultiResolutionImageReader()
image = reader.open(IMAGE_PATH)
dimensions = image.getDimensions()
spacing = image.getSpacing()

print("init time", time.time()-t1_init)
print("spacing level 0:", spacing)

t1_sampling = time.time()
for y in range(0, dimensions[1], TILE_SIZE):
    for x in range(0, dimensions[0], TILE_SIZE):
        image_tile = image.getUCharPatch(startX=x, startY=y, width=TILE_SIZE, height=TILE_SIZE, level=LEVEL)
print("sampling time", time.time()-t1_sampling)

init time 0.002272367477416992
spacing level 0: (0.5054008297457606, 0.5054008297457606)
sampling time 8.706735849380493


## WholeSlideData (4 CPUs)

In [4]:
CPUS = 4
t1_init = time.time()
patch_iterator = create_patch_iterator(image_path=IMAGE_PATH, tile_shape=(TILE_SIZE,TILE_SIZE,3), spacing=SPACING, cpus=CPUS)
print("init time", time.time()-t1_init)
t1_sampling = time.time()
for idx, (patch, info) in enumerate(patch_iterator):
    pass
print("sampling time", time.time()-t1_sampling )
patch_iterator.stop()

init time 0.14844727516174316
sampling time 2.936852216720581


## WholeSlideData (8 CPUs)

In [5]:
CPUS = 8
t1_init = time.time()
patch_iterator = create_patch_iterator(image_path=IMAGE_PATH,tile_shape=(TILE_SIZE,TILE_SIZE,3), spacing=SPACING, cpus=CPUS)
print("init time", time.time()-t1_init)
t1_sampling  = time.time()
for idx, (patch, info) in enumerate(patch_iterator):
    pass
print("sampling time", time.time()-t1_sampling )
patch_iterator.stop()

init time 0.22319579124450684
sampling time 1.764139175415039


## WholeSlideData (16 CPUs)

In [6]:
CPUS = 16
t1_init = time.time()
patch_iterator = create_patch_iterator(image_path=IMAGE_PATH, tile_shape=(TILE_SIZE,TILE_SIZE,3), spacing=SPACING, cpus=CPUS)
print("init time", time.time()-t1_init)
t1_sampling = time.time()
for idx, (patch, info) in enumerate(patch_iterator):
    pass
print("sampling time", time.time()-t1_sampling)
patch_iterator.stop()

init time 0.31551432609558105
sampling time 1.5458989143371582
